# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering Best Practices: Chain of Thought Reasoning </center>

# <center style="font-family: consolas; font-size: 25px; font-weight: bold;">  Prompt Engineering for Instruction-Tuned LLMs </center>
***

Prompt engineering, a fundamental concept in AI development, involves crafting tailored instructions or queries to guide AI models in generating desired outputs effectively across diverse tasks and scenarios. 

The notebook introduces the Chain of Thought Reasoning technique, which systematically guides AI models through step-by-step reasoning processes. This approach breaks down complex problems into manageable steps, enabling models to produce more accurate and coherent responses by considering various reasoning paths.

This comprehensive notebook covers the rationale behind using Chain of Thought Reasoning, practical examples demonstrating its application, guidelines for prompt structuring, and handling diverse user queries effectively. Additionally, it introduces the Inner Monologue concept for privacy preservation and recommends experimenting with prompt complexity to find the optimal balance between effectiveness and simplicity.

By exploring the strategies outlined in the notebook, readers can enhance the accuracy and coherence of AI-generated responses, improve user interactions, and safeguard privacy. Implementing the principles of prompt engineering and Chain of Thought Reasoning enables developers to create more efficient AI models, providing structured and informative interactions while optimizing user experience.

#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Introducing Chain of Thought Reasoning </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Setting Up Working Environment </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Chain of Thought Reasoning Practical Example </a></li> 
    <li><a href="#4" target="_self" rel=" noreferrer nofollow">4. Inner Monologue Removal </a></li>     
</ul>
</div>

***




<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Introducing Chain of Thought Reasoning </b></div>


Sometimes the model needs to reason in detail about a problem before answering a specific question. The model might make reasoning errors by rushing to an incorrect conclusion. Therefore we might need to reframe the query to request a series of relevant reasoning steps before the model provides a final answer.

By doing this it can think longer and more methodically about the problem and in general we call the strategy of asking the model to reason about a problem in steps **Chain of Thought reasoning**.
For some applications the reasoning process that our model uses to arrive at a final answer would be inappropriate to share with the user for example in tutoring applications we may want to encourage students to work on their answers but a model's reasoning process about the student solution could reveal the answer to the student in a **monologue**.

This  is a tactic that we will show by the end of the notebook that can be used to mitigate this and this is just a fancy way of hiding the model's reasoning from the user. The idea of the **inner monologue** is to instruct the model to put parts of the output that are meant to be hidden from the user into a structured format that makes passing them easy then before presenting the output to the user the output is pass
ed and only part of the output is made visible.

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Setting Up the Working Environment </b></div>


Let's initialize our working environment as usual by importing the OpenAI package and then define the API key from the local **.env** file:


In [1]:
!pip install openai

from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 10.5 MB/s eta 0:00:00


Now let's define a function that will take the prompt and return the response from the LLM.

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Chain of Thought Reasoning Practical Example </b></div>


Let's take a scenario in which we want the model to classify a customer query into a primary and secondary category and based on this  classification we might want to give the model different instructions. So if the product information category is in the next instructions we want to include information about this product.
Let's dive into this example and we will start with the delimiters that we will be using:


In [3]:
delimiter = "####"

Let's define  the  system message in which we will ask the model to reason about the answer before coming to its conclusion. So the instruction is to follow these steps to answer the customer queries the customer query will be delimited with four hashtags or delimiter.

Next, we will split the instructions into five steps. The first step is to decide whether the user is asking a question about a specific product or product category doesn't count.
Step two identify whether the products the user is asking about are in the following list and we will include a list of available products. Here we have five available products that are all varieties of laptops and these are all generated by GPT 4.

Then we will go to step three if the user is asking about specific products that are available in the above list. We will ask the LLM to list above any assumptions that the user is making in their message for example that laptop X is bigger than laptop Y or that laptop Z has a two-year warranty.

Step four is if the user made any assumptions figure out whether the assumption is true based on your product information

In step five we will ask the LLM to politely correct the customer's incorrect assumptions if applicable only mention or reference products in the list of the five available products as these are the only five products that the store sells and answer the customer in a friendly tone.

Finally, we will ask the model to use the following format step that will make it easier for us later to get just this response to the customer. Now let's put these whole steps together to create the final prompt

In [4]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

Now let's try an example in which the input will be "**By how much is the Blue Wave Chromebook more expensive than the Tech Pro desktop**" So let's take a look at these two products the Blue Wave Chromebook is 249.99 and the Tech Pro desktop is 999.99. 

This is not true and it will be interesting to see how the model will handle this user request. We will format it into our messages array and we'll get our response and then we'll print it. We are hoping that the model takes all of these different steps and realizes that the user has made an incorrect assumption and then follows the final step to politely correct the user.

So within this one prompt, we've maintained several different complex states that the system could be in so you know at any given point there could be a different output from the previous step and we would want to do something different for example if the user hadn't made any assumptions in step three then in step four we wouldn't have any output so this is a pretty complicated instruction for the model so let's see if it did it right.

In [5]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is comparing the prices of two specific products.
Step 2:#### The products being compared are:
- BlueWave Chromebook: Price $249.99
- TechPro Desktop: Price $999.99
Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The TechPro Desktop is actually more expensive than the BlueWave Chromebook by $750.
Response to user:#### The BlueWave Chromebook is $750 cheaper than the TechPro Desktop.


In step one the user asked a question about specific products. They're asking about the price difference between these two products the user assumes that the Blue Wave Chromebook is more expensive than the Techbook Pro and this assumption is incorrect. 

The LLM reasoning is taking longer to think about the problem in the same way that a human would also take some time to reason about an answer to any given question. 
The model performs better if it also has time to think and so the final response to the user is the Blue Wave Chromebook it's less expensive than the TechBook Pro which costs 999.99 while the Blue Wave Chromebook costs 249.99. 

Let's take another example of a user message in which the user will ask **"Do you sell TVs?"** If you remember in our product list we've only listed different computers so let's see what the model says in this case.

In [6]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a general question about whether the store sells TVs, not a specific product question. Therefore, the user is not asking about a specific product.


In step one the user will ask if the store sells TVs but since TVs are not listed in the available products you can see the model then skip the response to the user step because it realizes that the intermediary steps are not necessary.

I will say that we did ask for the output in this specific format so technically the model hasn't exactly followed our request again more advanced models will be better at doing that and so in this case our response to the user is "I'm sorry but we do not sell TVs at the store" and then it lists the available products.

<a id="4"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 4. Inner Monologue Removal </b></div>


If  we do not want to show the earlier parts of the response to the user we can just cut the string at the last occurrence of this delimiter token or string of four hashtags and then only print the final part of the model output. 

Here is the code to get only the final part of this string. We will use a try accept block to gracefully handle errors in case the model has some unpredictable output and doesn't use these characters. We are going to split the string at the delimiter string because we just want to get the last item in the output list then we're going to strip any white space because as you can see there might be white space after the characters then we're going to catch any errors and have a fallback response which is "**Sorry, I'm having trouble right now please try asking another question"**

In [7]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

The user is asking a general question about whether the store sells TVs, not a specific product question. Therefore, the user is not asking about a specific product.


If we were to integrate this prompt into an application, it's worth noting that this prompt is convoluted for the task at hand. You may find that you don't require all of these intermediary steps. 
I encourage you to explore alternative approaches to achieve the same goal more efficiently. Generally, finding the optimal balance in prompt complexity necessitates some experimentation. It's beneficial to try out several variations before settling on one.

# <div style="box-shadow: rgba(240, 46, 170, 0.4) -5px 5px inset, rgba(240, 46, 170, 0.3) -10px 10px inset, rgba(240, 46, 170, 0.2) -15px 15px inset, rgba(240, 46, 170, 0.1) -20px 20px inset, rgba(240, 46, 170, 0.05) -25px 25px inset; padding:20px; font-size:30px; font-family: consolas; display:fill; border-radius:15px; color: rgba(240, 46, 170, 0.7)"> <b> ༼⁠ ⁠つ⁠ ⁠◕⁠‿⁠◕⁠ ⁠༽⁠つ Thank You!</b></div>

<p style="font-family:verdana; color:rgb(34, 34, 34); font-family: consolas; font-size: 16px;"> 💌 Thank you for taking the time to read through my notebook. I hope you found it interesting and informative. If you have any feedback or suggestions for improvement, please don't hesitate to let me know in the comments. <br><br> 🚀 If you liked this notebook, please consider upvoting it so that others can discover it too. Your support means a lot to me, and it helps to motivate me to create more content in the future. <br><br> ❤️ Once again, thank you for your support, and I hope to see you again soon!</p>